In [27]:
import pandas as pd
import numpy as np
import itertools
from scipy.stats import fisher_exact
import shelve
import pickle
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from PIL import Image
from scipy.interpolate import BSpline, make_interp_spline, interp1d
import csv
from dfply import *
import itertools
import os
import logging
from sys import getsizeof
import sklearn
import time
from sklearn.metrics import roc_auc_score
from catboost import Pool, cv
import xgboost
import catboost
import scipy.stats as st

import importlib
from joblib import Parallel, delayed
from joblib import parallel_backend

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import lag_plot
#from statsmodels.tsa.vector_ar.var_model import VARY
from statsmodels.tsa.stattools import adfuller, kpss, grangercausalitytests, pacf
from statsmodels.graphics.tsaplots import plot_pacf
import warnings
warnings.filterwarnings("ignore")
import scipy
import datetime

from joblib import Parallel, delayed

import pyarrow
import concurrent.futures
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
from itertools import product

import statsmodels

In [28]:
datafolder = '/home/hoyinchan/blue/Data/data2022/'
labX4 = pd.read_parquet(datafolder+'allmedlabxx.parquet')

In [29]:
labX = pd.read_parquet(datafolder+'alllab.parquet')

In [30]:
labX4['LAB_LOINC'].unique()

array(['LG6657-3', 'LG49864-8', 'LG49936-4', 'LG11363-5'], dtype=object)

In [31]:
dummy_table = labX4[['ID', 'MEDADMIN_CODE_NEW']].drop_duplicates()

In [33]:
confound_med_HK =    {'MED:ATC:C03DA': 'eplerenone(HK):C03DA'}
confound_med_LK =    {'MED:ATC:A07DA': 'DiphenoxylateLoperamide(LK):A07DA'}
confound_med_HCa =    {'MED:ATC:M05BA': 'PamidronateZoledronate(HCa):M05BA'}
confound_med_LNa =    {'MED:ATC:C03CA': 'FurosemideBumetanideTorsemide(LKLNa):C03CA',
                      'MED:ATC:C03AA': 'hydrochlorothiazide(LNa):C03AA',
                      'MED:ATC:C03BA': 'chlorthalidone(LNa):C03BA',
                      'MED:ATC:C03XA': 'Vasopressin antagonists(LNa):C03XA'}

In [34]:
loincs_cofounder = ['LG6033-7', #AST       
                    'LG49755-8', #uric acid
                    '2532-0'] #LDH

loincs_dependents = ['LG49864-8', #Ca
                     'LG11363-5', #Na
                     'LG49936-4', 'LG6657-3'] #K, $sCr

In [35]:
dummy_table['if_confound_med_HK'] = dummy_table['MEDADMIN_CODE_NEW'].str.contains('|'.join(list(confound_med_HK.keys())))
dummy_table['if_confound_med_LK'] = dummy_table['MEDADMIN_CODE_NEW'].str.contains('|'.join(list(confound_med_LK.keys())))
dummy_table['if_confound_med_HCa'] = dummy_table['MEDADMIN_CODE_NEW'].str.contains('|'.join(list(confound_med_HCa.keys())))
dummy_table['if_confound_med_LNa'] =  dummy_table['MEDADMIN_CODE_NEW'].str.contains('|'.join(list(confound_med_LNa.keys())))

In [36]:
dummy_table = dummy_table.drop('MEDADMIN_CODE_NEW',axis=1).groupby('ID').sum().astype(bool).reset_index()

In [37]:
dummy_table = dummy_table.merge(labX[['ID']].drop_duplicates(), on='ID',how='outer').fillna(False)

In [38]:
K_bound =  [3.2, 4.4] # 2823-3
Na_Bound = [132, 144] # 2951-2
Ca_Bound = [8, 9.3] # 17861-6

In [39]:
dummy_table = dummy_table.merge(labX[labX['LAB_LOINC']=='LG49936-4'][['ID', 'DAYS_SINCE_ADMIT', 'RESULT_NUM']].groupby(['ID', 'DAYS_SINCE_ADMIT']).mean().reset_index().sort_values('DAYS_SINCE_ADMIT').groupby('ID').last().reset_index().drop('DAYS_SINCE_ADMIT',axis=1), on='ID', how='outer')
dummy_table = dummy_table.rename({'RESULT_NUM':'last_K'},axis=1)
dummy_table = dummy_table.merge(labX[labX['LAB_LOINC']=='LG11363-5'][['ID', 'DAYS_SINCE_ADMIT', 'RESULT_NUM']].groupby(['ID', 'DAYS_SINCE_ADMIT']).mean().reset_index().sort_values('DAYS_SINCE_ADMIT').groupby('ID').last().reset_index().drop('DAYS_SINCE_ADMIT',axis=1), on='ID', how='outer')
dummy_table = dummy_table.rename({'RESULT_NUM':'last_Na'},axis=1)
dummy_table = dummy_table.merge(labX[labX['LAB_LOINC']=='LG49864-8'][['ID', 'DAYS_SINCE_ADMIT', 'RESULT_NUM']].groupby(['ID', 'DAYS_SINCE_ADMIT']).mean().reset_index().sort_values('DAYS_SINCE_ADMIT').groupby('ID').last().reset_index().drop('DAYS_SINCE_ADMIT',axis=1), on='ID', how='outer')
dummy_table = dummy_table.rename({'RESULT_NUM':'last_Ca'},axis=1)
dummy_table = dummy_table.merge(labX[labX['LAB_LOINC']=='LG6657-3'][['ID', 'DAYS_SINCE_ADMIT', 'RESULT_NUM']].groupby(['ID', 'DAYS_SINCE_ADMIT']).mean().reset_index().sort_values('DAYS_SINCE_ADMIT').groupby('ID').last().reset_index().drop('DAYS_SINCE_ADMIT',axis=1), on='ID', how='outer')
dummy_table = dummy_table.rename({'RESULT_NUM':'last_sCr'},axis=1)

In [40]:
dummy_table['low_K'] = dummy_table['last_K'] <= K_bound[0]
dummy_table['high_K'] = dummy_table['last_K'] >= K_bound[1]
dummy_table['low_Na'] = dummy_table['last_Na'] <= Na_Bound[0]
dummy_table['high_Na'] = dummy_table['last_Na'] >= Na_Bound[1]
dummy_table['low_Ca'] = dummy_table['last_Ca'] <= Ca_Bound[0]
dummy_table['high_Ca'] = dummy_table['last_Ca'] >= Ca_Bound[1]

In [41]:
#dummy_table = dummy_table.drop(['last_K','last_Na', 'last_Ca'],axis=1)

In [42]:
final_table = labX[labX['LAB_LOINC'].str.contains('|'.join(loincs_cofounder+loincs_dependents)).fillna(False)][['ID', 'DAYS_SINCE_ADMIT', 'LAB_LOINC', 'RESULT_NUM']]

In [43]:
final_table = final_table.groupby(['ID', 'DAYS_SINCE_ADMIT', 'LAB_LOINC']).mean().reset_index()

In [44]:
loincs_cofounder+loincs_dependents

['LG6033-7',
 'LG49755-8',
 '2532-0',
 'LG49864-8',
 'LG11363-5',
 'LG49936-4',
 'LG6657-3']

In [45]:
final_table['LAB_LOINC'].unique()

array(['LG11363-5', 'LG49864-8', 'LG49936-4', 'LG6657-3', 'LG6033-7',
       'LG49755-8', '2532-0'], dtype=object)

In [46]:
pivot_table = final_table.pivot_table(index=['ID', 'DAYS_SINCE_ADMIT'], 
                                      columns='LAB_LOINC', 
                                      values='RESULT_NUM').reset_index()

In [47]:
pivot_table = pivot_table.merge(dummy_table, on='ID', how='left')

In [48]:
pivot_table

,ID,DAYS_SINCE_ADMIT,2532-0,LG11363-5,LG49755-8,LG49864-8,LG49936-4,LG6033-7,LG6657-3,if_confound_med_HK,...,last_K,last_Na,last_Ca,last_sCr,low_K,high_K,low_Na,high_Na,low_Ca,high_Ca
0,0000f45b1ab3b10c_8fccdcecca9d71a2_MCW,0,NaN,136.0,NaN,10.0,3.3,NaN,0.52,False,...,3.9,137.0,9.4,0.56,False,False,False,False,False,True
1,0000f45b1ab3b10c_8fccdcecca9d71a2_MCW,1,NaN,138.0,NaN,9.2,3.5,NaN,0.58,False,...,3.9,137.0,9.4,0.56,False,False,False,False,False,True
2,0000f45b1ab3b10c_8fccdcecca9d71a2_MCW,2,NaN,139.0,NaN,9.3,3.5,NaN,0.62,False,...,3.9,137.0,9.4,0.56,False,False,False,False,False,True
3,0000f45b1ab3b10c_8fccdcecca9d71a2_MCW,3,NaN,137.0,NaN,9.4,3.9,NaN,0.56,False,...,3.9,137.0,9.4,0.56,False,False,False,False,False,True
4,0001dfea5dd5c942_e8e082346f6b6c40_MCW,0,NaN,120.0,NaN,8.1,3.7,34.0,0.86,False,...,3.1,120.0,7.1,0.44,True,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395925,fffad3b37903dcfd_b1b8018bf9ac5f71_MCW,0,NaN,143.0,NaN,9.3,4.1,22.0,0.61,False,...,4.1,143.0,8.0,0.40,False,False,False,False,True,False
6395926,fffad3b37903dcfd_b1b8018bf9ac5f71_MCW,1,NaN,142.0,NaN,8.2,3.2,NaN,0.41,False,...,4.1,143.0,8.0,0.40,False,False,False,False,True,False
6395927,fffad3b37903dcfd_b1b8018bf9ac5f71_MCW,2,NaN,143.0,NaN,8.0,2.9,11.0,0.38,False,...,4.1,143.0,8.0,0.40,False,False,False,False,True,False
6395928,fffad3b37903dcfd_b1b8018bf9ac5f71_MCW,3,NaN,NaN,NaN,NaN,NaN,NaN,0.40,False,...,4.1,143.0,8.0,0.40,False,False,False,False,True,False


In [49]:
pivot_table['ID2'] = pd.factorize(pivot_table['ID'])[0]

In [50]:
pivot_table[['if_confound_med_LNa', 'if_confound_med_HCa', 'if_confound_med_HK', 'if_confound_med_LK', 'low_K', 'high_K', 'low_Na', 'high_Na', 'low_Ca', 'high_Ca']] = pivot_table[['if_confound_med_LNa', 'if_confound_med_HCa', 'if_confound_med_HK', 'if_confound_med_LK', 'low_K', 'high_K', 'low_Na', 'high_Na', 'low_Ca', 'high_Ca']].astype(int)

In [51]:
pivot_table

,ID,DAYS_SINCE_ADMIT,2532-0,LG11363-5,LG49755-8,LG49864-8,LG49936-4,LG6033-7,LG6657-3,if_confound_med_HK,...,last_Na,last_Ca,last_sCr,low_K,high_K,low_Na,high_Na,low_Ca,high_Ca,ID2
0,0000f45b1ab3b10c_8fccdcecca9d71a2_MCW,0,NaN,136.0,NaN,10.0,3.3,NaN,0.52,0,...,137.0,9.4,0.56,0,0,0,0,0,1,0
1,0000f45b1ab3b10c_8fccdcecca9d71a2_MCW,1,NaN,138.0,NaN,9.2,3.5,NaN,0.58,0,...,137.0,9.4,0.56,0,0,0,0,0,1,0
2,0000f45b1ab3b10c_8fccdcecca9d71a2_MCW,2,NaN,139.0,NaN,9.3,3.5,NaN,0.62,0,...,137.0,9.4,0.56,0,0,0,0,0,1,0
3,0000f45b1ab3b10c_8fccdcecca9d71a2_MCW,3,NaN,137.0,NaN,9.4,3.9,NaN,0.56,0,...,137.0,9.4,0.56,0,0,0,0,0,1,0
4,0001dfea5dd5c942_e8e082346f6b6c40_MCW,0,NaN,120.0,NaN,8.1,3.7,34.0,0.86,0,...,120.0,7.1,0.44,1,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395925,fffad3b37903dcfd_b1b8018bf9ac5f71_MCW,0,NaN,143.0,NaN,9.3,4.1,22.0,0.61,0,...,143.0,8.0,0.40,0,0,0,0,1,0,1325636
6395926,fffad3b37903dcfd_b1b8018bf9ac5f71_MCW,1,NaN,142.0,NaN,8.2,3.2,NaN,0.41,0,...,143.0,8.0,0.40,0,0,0,0,1,0,1325636
6395927,fffad3b37903dcfd_b1b8018bf9ac5f71_MCW,2,NaN,143.0,NaN,8.0,2.9,11.0,0.38,0,...,143.0,8.0,0.40,0,0,0,0,1,0,1325636
6395928,fffad3b37903dcfd_b1b8018bf9ac5f71_MCW,3,NaN,NaN,NaN,NaN,NaN,NaN,0.40,0,...,143.0,8.0,0.40,0,0,0,0,1,0,1325636


In [53]:
# loinc_to_label = {
#     '1920-8': 'AST',
#     '3084-1': 'UA',
#     '17861-6': 'Ca',
#     '2951-2': 'Na',
#     '2823-3': 'K',
#     '2160-0': 'sCr'
# }

loinc_to_label = {
    '2532-0': 'LDH',
    'LG6033-7': 'AST',
    'LG49755-8': 'UA',
    'LG49864-8': 'Ca',
    'LG11363-5': 'Na',
    'LG49936-4': 'K',
    'LG6657-3': 'sCr'
}

pivot_table = pivot_table.rename(columns=loinc_to_label)

In [54]:
pivot_table.to_parquet(datafolder+'confounder_regression.parquet')

In [55]:
pivot_table.columns

Index(['ID', 'DAYS_SINCE_ADMIT', 'LDH', 'Na', 'UA', 'Ca', 'K', 'AST', 'sCr',
       'if_confound_med_HK', 'if_confound_med_LK', 'if_confound_med_HCa',
       'if_confound_med_LNa', 'last_K', 'last_Na', 'last_Ca', 'last_sCr',
       'low_K', 'high_K', 'low_Na', 'high_Na', 'low_Ca', 'high_Ca', 'ID2'],
      dtype='object')

In [56]:
pivot_table.columns

Index(['ID', 'DAYS_SINCE_ADMIT', 'LDH', 'Na', 'UA', 'Ca', 'K', 'AST', 'sCr',
       'if_confound_med_HK', 'if_confound_med_LK', 'if_confound_med_HCa',
       'if_confound_med_LNa', 'last_K', 'last_Na', 'last_Ca', 'last_sCr',
       'low_K', 'high_K', 'low_Na', 'high_Na', 'low_Ca', 'high_Ca', 'ID2'],
      dtype='object')